In [12]:
import sys,os
import pickle
import anndata
import pandas as pd
import numpy as np
import scanpy as sc
import pickle
import matplotlib.pyplot as plt

In [2]:
adata = sc.read_h5ad(open("/nfs/team283/aa16/KR_NAS/data/CR_cells_AnnData_1.h5ad", "rb"))

In [3]:
adata = adata[adata.obs['Pool'] == 'CRcells',:]

In [4]:
adata.obs['GeoLOD2.0_01'] = adata.obs['NegGeoMean_01'] * adata.obs['NegGeoSD_01']**2

Trying to set attribute `.obs` of view, copying.


In [5]:
adata.obs['Donor'] = [x.split('-')[0] for x in adata.obs['Sample slide ID']]

In [8]:
unique_donors = np.unique(adata.obs['Donor'])
detection_list = []
nuclei_list = []
for j in range(len(unique_donors)):
    subset = [adata.obs['Donor'][i] == unique_donors[j] for i in range(len(adata.obs['Donor']))]
    adata_subset = adata[subset,:]
    detection_list.append([sum(adata_subset.X[i,:] > adata_subset.obs['GeoLOD2.0_01'].iloc[i]) for i in range(np.shape(adata_subset)[0])])
    nuclei_list.append(np.array(adata_subset.obs['nuclei']))

In [18]:
donor = ('Donor 1', 'Donor 2')
colours = ('Red', 'Blue', 'Yellow', 'Grey')

SMALL_SIZE = 18
MEDIUM_SIZE = 18
BIGGER_SIZE = 18

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=14)  # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

plt.figure(figsize=(4,6))

count = 0
for j in range(2):
        plt.scatter(nuclei_list[count], detection_list[count], c = colours[j],
                    label = donor[j],
                    s = 50)
        plt.legend(loc = 'lower right')
        plt.ylabel('Number of detected genes')
        plt.xlabel('Number of nuclei')
        count += 1

plt.savefig('/nfs/team283/aa16/KR_NAS/0_publication_figures/Figure5_DetectedGenesPerAOI_vsNuclei_LoD20.pdf',
            bbox_inches="tight")